In [ ]:
# Copy and paste all required information here for the listed variables, make sure to paste inside the quotation marks

# Your GCS bucket name that resides in the GCS project
GCS_BUCKET_NAME = '' 

# The file name where you saved your GCS credentials, be sure to include .json
GCS_CREDS_JSON = '' 

# The order id that was generated when the subscription request was created
ORDER_ID = ''

# set email reciever and sender with secure app password
MAIL_FROM = ''
SECURE_APP_PASSWORD = ''
MAIL_TO = ''

# set threshold of area covered or percentage of area covered can leave as preset values 
max_change_percent = 5      
max_change_area_m2 = 100   

In [ ]:
# import packages 

import os
import sys

import rasterio
from rasterio.plot import show
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image


import skimage as ski
from skimage.color import rgb2gray
from skimage import data
from skimage.filters import gaussian
from skimage.segmentation import active_contour
from skimage.draw import polygon
from skimage import measure



from scipy.signal import convolve2d as conv2
from skimage import color, data, restoration
from copy import copy

from google.cloud import storage

import email
import smtplib
import ssl

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:
# collecting info on the google cloud storage space

# bucket prefix info for specific subscription
prefix = ORDER_ID + '/'

# set up directory - for current file location
cwd = os.getcwd()
folder = cwd + '/' + prefix

# Create a client object. The client can be reused for multiple calls.
# location of the storage code in the same folder
storage_client = storage.Client.from_service_account_json(GCS_CREDS_JSON)

# Get the bucket that the file will be downloaded from.
bucket = storage_client.get_bucket(GCS_BUCKET_NAME)


In [ ]:
# create a folder to store images

#  Create this folder locally if not exists
if not os.path.exists(folder):
  os.makedirs(folder)
  #print('Folder was created')
#else:
  #print('Folder already exists')

# create sub folder to store comparision images/ tables
sub_folder_comp_images = folder + 'Comparison Images/'
if not os.path.exists(sub_folder_comp_images):
    os.makedirs(sub_folder_comp_images)
    #print('Sub-Folder was created')
#else:
    #print('Sub-Folder already exists')

In [ ]:
# downloads all non repeat days to folder from google cloud storage if not already downloaded

blobs = bucket.list_blobs(prefix=prefix)

previous_date = 00000000
previous_id = 00000000000000

# empty arrarys that are filled to store the files names of each type of file retrieved
analytic_files = []
mask_files = []
metadata_files = []
harm_metadata_files = []

# keep track if a new image has been downloaded
new_image_downloaded = False

for blob in blobs:
  filename = blob.name

  filename = filename.split('/')[-1]
  #print(filename)

  date = filename[0:8]


  id = filename[9:23]

  file_type = filename[24:]



  if not os.path.exists(folder + filename):
    if previous_date != date:
      blob.download_to_filename(folder + filename)
      #print('File downloaded')
      new_image_downloaded = True
      previous_date = date
      previous_id = id
      if file_type == '3B_AnalyticMS_SR_harmonized_clip.tif': 
        analytic_files.append(filename)
      elif file_type == '3B_udm2_clip.tif':
        mask_files.append(filename)
      elif file_type == 'metadata.json':
        metadata_files.append(filename)
      elif file_type == '3B_AnalyticMS_metadata_clip.xml': 
        harm_metadata_files.append(filename)
    elif previous_date == date and previous_id == id:
      blob.download_to_filename(folder + filename)
      #print('File downloaded')
      new_image_downloaded = True
      if file_type == '3B_AnalyticMS_SR_harmonized_clip.tif': 
        analytic_files.append(filename)
      elif file_type == '3B_udm2_clip.tif':
        mask_files.append(filename)
      elif file_type == 'metadata.json':
        metadata_files.append(filename)
      elif file_type == '3B_AnalyticMS_metadata_clip.xml': 
        harm_metadata_files.append(filename)
    #else:
      #print('Multiple images on the same day - file not downloaded')
  else:
    if previous_date != date:
      previous_date = date
      previous_id = id
    #print('File already downloaded')
    if file_type == '3B_AnalyticMS_SR_harmonized_clip.tif': 
      analytic_files.append(filename)
    elif file_type == '3B_udm2_clip.tif':
      mask_files.append(filename)
    elif file_type == 'metadata.json':
      metadata_files.append(filename)
    elif file_type == '3B_AnalyticMS_metadata_clip.xml':  
        harm_metadata_files.append(filename)

  #print()



#print('Total Images: ', len(analytic_files))

# stops code from running if no new images were downlaoded since the last time it was run
if new_image_downloaded == False:
    #print('No new images')
    sys.exit()

#print('New Image Downlaoded: ', new_image_downloaded)

In [ ]:
# loads current image and images from 5 instances before that
# only runs if there are at least 6 images downloaded

if len(analytic_files) >= 6:
  current_image = rasterio.open(folder + analytic_files[-1])
  current_image_name = analytic_files[-1].split('_')[0]
  current_image_name = current_image_name[4:6] + '-' + current_image_name[6:] + '-' + current_image_name[0:4] +  ' Image'
  current_image_mask = rasterio.open(folder + mask_files[-1])
  current_image_mask_name = mask_files[-1].split('_')[0]
  current_image_mask_name = current_image_mask_name[4:6] + '-' + current_image_mask_name[6:] + '-' + current_image_mask_name[0:4] +  ' Mask'

  past_image_1 = rasterio.open(folder + analytic_files[-2])
  past_image_1_name = analytic_files[-2].split('_')[0]
  past_image_1_name = past_image_1_name[4:6] + '-' + past_image_1_name[6:] + '-' + past_image_1_name[0:4] +  ' Image'
  past_image_1_mask = rasterio.open(folder + mask_files[-2])
  past_image_1_mask_name = mask_files[-2].split('_')[0]
  past_image_1_mask_name = past_image_1_mask_name[4:6] + '-' + past_image_1_mask_name[6:] + '-' + past_image_1_mask_name[0:4] +  ' Mask'


  past_image_2 = rasterio.open(folder + analytic_files[-3])
  past_image_2_name = analytic_files[-3].split('_')[0]
  past_image_2_name = past_image_2_name[4:6] + '-' + past_image_2_name[6:] + '-' + past_image_2_name[0:4] +  ' Image'
  past_image_2_mask = rasterio.open(folder + mask_files[-3])
  past_image_2_mask_name = mask_files[-3].split('_')[0]
  past_image_2_mask_name = past_image_2_mask_name[4:6] + '-' + past_image_2_mask_name[6:] + '-' + past_image_2_mask_name[0:4] +  ' Mask'


  past_image_3 = rasterio.open(folder + analytic_files[-4])
  past_image_3_name = analytic_files[-4].split('_')[0]
  past_image_3_name = past_image_3_name[4:6] + '-' + past_image_3_name[6:] + '-' + past_image_3_name[0:4] +  ' Image'
  past_image_3_mask = rasterio.open(folder + mask_files[-4])
  past_image_3_mask_name = mask_files[-4].split('_')[0]
  past_image_3_mask_name = past_image_3_mask_name[4:6] + '-' + past_image_3_mask_name[6:] + '-' + past_image_3_mask_name[0:4] +  ' Mask'


  past_image_4 = rasterio.open(folder + analytic_files[-5])
  past_image_4_name = analytic_files[-5].split('_')[0]
  past_image_4_name = past_image_4_name[4:6] + '-' + past_image_4_name[6:] + '-' + past_image_4_name[0:4] +  ' Image'
  past_image_4_mask = rasterio.open(folder + mask_files[-5])
  past_image_4_mask_name = mask_files[-5].split('_')[0]
  past_image_4_mask_name = past_image_4_mask_name[4:6] + '-' + past_image_4_mask_name[6:] + '-' + past_image_4_mask_name[0:4] +  ' Mask'


  past_image_5 = rasterio.open(folder + analytic_files[-6])
  past_image_5_name = analytic_files[-6].split('_')[0]
  past_image_5_name = past_image_5_name[4:6] + '-' + past_image_5_name[6:] + '-' + past_image_5_name[0:4] +  ' Image'
  past_image_5_mask = rasterio.open(folder + mask_files[-6])
  past_image_5_mask_name = mask_files[-6].split('_')[0]
  past_image_5_mask_name = past_image_5_mask_name[4:6] + '-' + past_image_5_mask_name[6:] + '-' + past_image_5_mask_name[0:4] +  ' Mask'

else:
    # stops code execution if there aren't 6 days worth of data downloaded yet
    print('Not enough image data')
    sys.exit()
    


images = [current_image, past_image_1, past_image_2, past_image_3, past_image_4, past_image_5]
image_names = [current_image_name, past_image_1_name, past_image_2_name, past_image_3_name, past_image_4_name, past_image_5_name]
masks = [current_image_mask, past_image_1_mask, past_image_2_mask, past_image_3_mask, past_image_4_mask, past_image_5_mask]
mask_names = [current_image_mask_name, past_image_1_mask_name, past_image_2_mask_name, past_image_3_mask_name, past_image_4_mask_name, past_image_5_mask_name]


In [ ]:
# normalize definition (0.0-1.0 scale)

def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

In [ ]:
# reads image and returns normalized and stacked rgb image

def stack_norm_img(image, image_name):
  b, g, r, n = image.read()

  red_norm = normalize(r)
  green_norm = normalize(g)
  blue_norm = normalize(b)
  nir_norm = normalize(n)

  rgb_norm = np.dstack((red_norm, green_norm, blue_norm))

  return rgb_norm


In [ ]:
# reads image and returns stacked rgb image

def stack_raw_img(image, image_name):
  b, g, r, n = image.read()
  rgb_raw = np.dstack((r, g, b))/16
  rgb_raw = rgb_raw.astype(int)
  rgb_raw = np.clip(rgb_raw, 0, 255)

  return rgb_raw

In [ ]:
# stacks bands for raw and normalized images and saves them in an array
raw_images = []
norm_images = []

for i in range(0,len(images)):
  norm_images.append(stack_norm_img(images[i], image_names[i]))
  raw_images. append(stack_raw_img(images[i], image_names[i]))

  # plt.subplot(121)
  # plt.imshow(norm_images[i])
  # plt.title('Normalized Image')
  # plt.subplot(122)
  # plt.imshow(raw_images[i])
  # plt.title('Raw Image')
  # plt.suptitle(image_names[i], y=0.8)
  # plt.show()
  # print('')
  # print('')
  # print('')
  # print('')

In [ ]:
# create clear mask

def load_clear_mask(mask, mask_name):
  clear_mask_bool = mask.read(1).astype(bool)
  confidence = mask.read(7).astype(int)
  unusable = mask.read(8)

  rows = mask.shape[0]
  cols = mask.shape[1]
  clear_mask_int = np.zeros((rows,cols), dtype='int')
  clear_mask_float = np.zeros((rows,cols), dtype='float64')

  clear_pixels = 0
  obscured_pixels = 0
  black_fill = 0

  for r in range(0,rows):
    for c in range(0,cols):
      # only say its clear if you are at least 95 percent sure that it is clear
      if clear_mask_bool[r,c] == True and confidence[r,c] > 95:
        clear_mask_int[r,c] = 1
        clear_mask_float[r,c] = 1
        clear_pixels = clear_pixels + 1
      elif unusable[r,c] == 1:
        # counts pixels that aren't part of AOI
        black_fill = black_fill + 1
        clear_mask_int[r,c] = 0
        clear_mask_float[r,c] = 0  
      else:
        clear_mask_int[r,c] = 0
        clear_mask_float[r,c] = 0
        obscured_pixels = obscured_pixels + 1

  total_pixels_usable = clear_pixels + obscured_pixels
  clear_percent = clear_pixels / total_pixels_usable

  return clear_mask_int, clear_mask_float, clear_percent, total_pixels_usable


In [ ]:
# create clear mask for all given masks
clear_masks_int = []
clear_masks_float = []
clear_percents = []

for i in range(0,len(masks)):
  clear_mask_int, clear_mask_float, clear_percent, usable_pixels = load_clear_mask(masks[i], mask_names[i])
  clear_masks_int.append(clear_mask_int)
  clear_masks_float.append(clear_mask_float)
  clear_percents.append(clear_percent)

  # plt.imshow(clear_mask_int, cmap='gray')
  # plt.title('Clear Mask ' + mask_names[i] + '\n White Indicates Clear Areas')
  # plt.show()

  # print('')
  # print('')
  # print('')
  # print('')


In [ ]:
# create masked images by multiplying clear mask by raw and normalized images
clear_mask_norm_images = []
clear_mask_raw_images = []

for i in range(0, len(masks)):
  clear_mask_norm_images.append(norm_images[i] * np.dstack((clear_masks_float[i], clear_masks_float[i], clear_masks_float[i])))
  clear_mask_raw_images.append(raw_images[i] * np.dstack((clear_masks_int[i], clear_masks_int[i], clear_masks_int[i])))

  # plt.subplot(121)
  # plt.imshow(clear_mask_norm_images[i])
  # plt.title('Masked Norm')
  # plt.subplot(122)
  # plt.imshow(clear_mask_raw_images[i])
  # plt.title('Masked Raw')
  # plt.suptitle(image_names[i], y=0.8)
  # plt.show()

  # print('')
  # print('')
  # print('')
  # print('')



In [ ]:
# get max differences for previous 5 images (raw and normalized)

rows = clear_mask_norm_images[0].shape[0]
cols = clear_mask_norm_images[0].shape[1]

max_diff_norm = np.zeros_like(clear_mask_norm_images[0])
max_diff_raw = np.zeros_like(clear_mask_raw_images[0])

for r in range(0, rows):
  for c in range(0, cols):
    # get max and min for raw images
    # need to make sure not to choose 0 as the min when its in a masked area

    # max, min, and diff for red for 5 previous raw images
    red_array_raw = [clear_mask_raw_images[1][r,c,0], clear_mask_raw_images[2][r,c,0], clear_mask_raw_images[3][r,c,0], clear_mask_raw_images[4][r,c,0], clear_mask_raw_images[5][r,c,0]]
    red_max_raw = max(red_array_raw)
    if red_max_raw == 0:
      red_min_raw = 0
    else:
      red_min_raw = min(x for x in red_array_raw if x!=0)
    max_diff_raw[r,c,0] = red_max_raw - red_min_raw

    # max, min, and diff for green for 5 previous raw images
    green_array_raw = [clear_mask_raw_images[1][r,c,1], clear_mask_raw_images[2][r,c,1], clear_mask_raw_images[3][r,c,1], clear_mask_raw_images[4][r,c,1], clear_mask_raw_images[5][r,c,1]]
    green_max_raw = max(green_array_raw)
    if green_max_raw == 0:
      green_min_raw = 0
    else:
      green_min_raw = min(x for x in green_array_raw if x!=0)
    max_diff_raw[r,c,1] = green_max_raw - green_min_raw

    # max, min, and diff for blue for 5 previous raw images
    blue_array_raw = [clear_mask_raw_images[1][r,c,2], clear_mask_raw_images[2][r,c,2], clear_mask_raw_images[3][r,c,2], clear_mask_raw_images[4][r,c,2], clear_mask_raw_images[5][r,c,2]]
    blue_max_raw = max(blue_array_raw)
    if blue_max_raw == 0:
      blue_min_raw = 0
    else:
      blue_min_raw = min(x for x in blue_array_raw if x!=0)
    max_diff_raw[r,c,2] = blue_max_raw - blue_min_raw


    # get max and min for norm images
    # need to make sure not to choose 0 as the min when its in a masked area

    # max, min, and diff for red for 5 previous norm images
    red_array_norm = [clear_mask_norm_images[1][r,c,0], clear_mask_norm_images[2][r,c,0], clear_mask_norm_images[3][r,c,0], clear_mask_norm_images[4][r,c,0], clear_mask_norm_images[5][r,c,0]]
    red_max_norm = max(red_array_norm)
    if red_max_norm == 0:
      red_min_norm = 0
    else:
      red_min_norm = min(x for x in red_array_norm if x!=0)
    max_diff_norm[r,c,0] = red_max_norm - red_min_norm

    # max, min, and diff for green for 5 previous norm images
    green_array_norm = [clear_mask_norm_images[1][r,c,1], clear_mask_norm_images[2][r,c,1], clear_mask_norm_images[3][r,c,1], clear_mask_norm_images[4][r,c,1], clear_mask_norm_images[5][r,c,1]]
    green_max_norm = max(green_array_norm)
    if green_max_norm == 0:
      green_min_norm = 0
    else:
      green_min_norm = min(x for x in green_array_norm if x!=0)
    max_diff_norm[r,c,1] = green_max_norm - green_min_norm

    # max, min, and diff for blue for 5 previous norm images
    blue_array_norm = [clear_mask_norm_images[1][r,c,2], clear_mask_norm_images[2][r,c,2], clear_mask_norm_images[3][r,c,2], clear_mask_norm_images[4][r,c,2], clear_mask_norm_images[5][r,c,2]]
    blue_max_norm = max(blue_array_norm)
    if blue_max_norm == 0:
      blue_min_norm = 0
    else:
      blue_min_norm = min(x for x in blue_array_norm if x!=0)
    max_diff_norm[r,c,2] = blue_max_norm - blue_min_norm


# show image of the max differences
# plt.subplot(121)
# plt.imshow(max_diff_norm)
# plt.title('Norm')
# plt.subplot(122)
# plt.imshow(max_diff_raw)
# plt.title('Raw')
# plt.suptitle('Max Differences Over 5 Days', y=0.8)
# plt.show()



In [ ]:
# compare the max differences from the previous 5 days to the current day's image

comp_raw = np.zeros_like(max_diff_raw)
comp_norm = np.zeros_like(max_diff_norm)
comp_raw_binary = np.zeros([rows,cols], dtype='int')
comp_raw_binary_mask = np.zeros([rows,cols], dtype='int')
comp_norm_binary = np.zeros([rows,cols])


raw_scale = 13 # 5% of pixel scale
norm_scale = 0.05 # 5% of pixel scale


#### check which previous day to compare to, only compare if current image and previous image overlap by 75% or more
# set the min overlap
min_overlap = 0.75
for i in range(0, (len(clear_mask_raw_images) - 1)):
  temp_image = clear_mask_raw_images[0] * clear_mask_raw_images[i+1]
  clear_overlap_pixels = 0
  for r in range(0,rows):
    for c in range(0,cols):
      if temp_image[r,c,0] != 0 and temp_image[r,c,1] != 0 and temp_image[r,c,2] != 0:
        clear_overlap_pixels = clear_overlap_pixels + 1
    overlap_percent = clear_overlap_pixels / usable_pixels

  if overlap_percent >= min_overlap:
    comparision_index = i+1
    break
  if i == (len(clear_mask_raw_images) - 2):
    # exits the script and comparision not run if not enough overlap
    print('Not enough clear overlap - comparision not run')
    sys.exit()



# count number of pixels effected
raw_diff_pixel_count = 0
norm_diff_pixel_count = 0


# compare todays image to the previous days image, see if the difference is greater than the 5 day max difference or 5% of the pixel range
# only make the comparision if both today and previous images are not equal to zero, if they are zero set to another color to show that that information was obscured

for r in range(0,rows):
  for c in range(0,cols):
    if clear_mask_raw_images[0][r,c,0] != 0 and clear_mask_raw_images[0][r,c,1] != 0 and clear_mask_raw_images[0][r,c,2] != 0 and clear_mask_raw_images[comparision_index][r,c,0] != 0 and clear_mask_raw_images[comparision_index][r,c,1] != 0 and clear_mask_raw_images[comparision_index][r,c,2] != 0:
      comp_raw[r,c,:] = abs(clear_mask_raw_images[0][r,c,:] - clear_mask_raw_images[comparision_index][r,c,:])
      comp_norm[r,c,:] = abs(clear_mask_norm_images[0][r,c,:] - clear_mask_norm_images[comparision_index][r,c,:])


      ##### need to check percent scale versus max diff at the location to ensure that we use the bigger number to compare to
      if max_diff_raw[r,c,0] > raw_scale:
        red_raw_scale = max_diff_raw[r,c,0]
      else:
        red_raw_scale = raw_scale

      if max_diff_raw[r,c,1] > raw_scale:
        green_raw_scale = max_diff_raw[r,c,1]
      else:
        green_raw_scale = raw_scale

      if max_diff_raw[r,c,2] > raw_scale:
        blue_raw_scale = max_diff_raw[r,c,2]
      else:
        blue_raw_scale = raw_scale

      if (comp_raw[r,c,0] > red_raw_scale) or (comp_raw[r,c,1] > green_raw_scale) or (comp_raw[r,c,2] > blue_raw_scale):
        comp_raw_binary[r,c] = 255
        comp_raw_binary_mask[r,c] = 255
        raw_diff_pixel_count = raw_diff_pixel_count + 1


      ##### need to check percent scale versus max diff at the location to ensure that we use the bigger number to compare to
      if max_diff_norm[r,c,0] > norm_scale:
        red_norm_scale = max_diff_norm[r,c,0]
      else:
        red_norm_scale = norm_scale

      if max_diff_norm[r,c,1] > norm_scale:
        green_norm_scale = max_diff_norm[r,c,1]
      else:
        green_norm_scale = norm_scale

      if max_diff_norm[r,c,2] > norm_scale:
        blue_norm_scale = max_diff_norm[r,c,2]
      else:
        blue_norm_scale = norm_scale

      if (comp_norm[r,c,0] > red_norm_scale) or (comp_norm[r,c,1] > green_norm_scale) or (comp_norm[r,c,2] > blue_norm_scale):
        comp_norm_binary[r,c] = 1
        norm_diff_pixel_count = norm_diff_pixel_count + 1

    else:
      comp_raw[r,c,:] = [128, 128, 128]
      comp_raw_binary[r,c] = 128
      comp_raw_binary_mask[r,c] = 0
      comp_norm[r,c,:] = [0.5, 0.5, 0.5]
      comp_norm_binary[r,c] = 0.5

    

effected_raw_percent = round(((raw_diff_pixel_count / usable_pixels) * 100), 2)
effected_norm_percent = round(((norm_diff_pixel_count / usable_pixels) * 100), 2)

# multiply number of pixels by the area of one pixel = 9 for 3m * 3m resolution
effected_raw_area = raw_diff_pixel_count * 9
effected_raw_area_acres = round((effected_raw_area / 4047), 2)
effected_norm_area = norm_diff_pixel_count * 9
effected_norm_area_acres = round((effected_norm_area / 4047), 2)

# find number of regions of difference and the areas of each region
input_mask = comp_raw_binary_mask.copy()/255
labels_mask = measure.label(input_mask)
regions = measure.regionprops(labels_mask)
regions.sort(key=lambda x: x.area, reverse=True)
region_areas = []
for rg in regions:
  region_areas.append(rg.area)


labels_mask[labels_mask!=0] = 1


largest_effected_percent = round(((region_areas[0] / usable_pixels) * 100), 2)
largest_effected_area = region_areas[0] * 9
largest_effected_area_acres = round((largest_effected_area / 4047), 2)

# print('Largest Effected Percent: '+ str(largest_effected_percent) + ' %' )
# print('Largest Effected Area: '+ str(largest_effected_area) + ' m^2' )
# print('Largest Effected Area: '+ str(largest_effected_area_acres) + ' acres' )

# print('')
# print('')

# plt.subplot(121)
# plt.imshow(comp_raw)
# plt.title('RGB')
# plt.subplot(122)
# plt.imshow(comp_raw_binary, cmap='gray')
# plt.title('Binary Threshold')
# plt.suptitle('Raw Image', y=0.8)
# plt.show()

# print('')
# print('')
# print('')
# print('')

# plt.subplot(121)
# plt.imshow(comp_norm)
# plt.title('RGB')
# plt.subplot(122)
# plt.imshow(comp_norm_binary, cmap='gray')
# plt.title('Binary Threshold')
# plt.suptitle('Norm Image', y=0.8)
# plt.show()

# print('')
# print('')

# print('Effected Raw Percent: ', effected_raw_percent, '%')
# print('Effected Raw Area: ', effected_raw_area, 'm^2')
# print('Effected Raw Area: ', effected_raw_area_acres, 'acres')
# print('')
# print('Effected Norm Percent: ', effected_norm_percent, '%')
# print('Effected Norm Area: ', effected_norm_area, 'm^2')
# print('Effected Norm Area: ', effected_norm_area_acres, 'acres')



In [ ]:
# overlay current norm image with raw binary image
overlay_image = clear_mask_norm_images[0] * np.dstack((comp_raw_binary_mask/255, comp_raw_binary_mask/255, comp_raw_binary_mask/255))
saved_image_dir = sub_folder_comp_images + image_names[0] + ' - Affected Area'

plt.ioff()
# create and save a file of the rgb norm image of the previous day, the current day, the binary difference threshold and the difference overlay
# include a summary table of results
plt.figure(figsize=(16,9))
plt.subplot(241)
plt.imshow(clear_mask_norm_images[comparision_index]) #was plt.imshow
plt.title('Previous Comparision Image\n' + image_names[comparision_index][0:10], y=1.1, fontsize=15)
plt.subplot(242)
plt.imshow(clear_mask_norm_images[0])
plt.title('Current Image\n' + image_names[0][0:10], y=1.1, fontsize=15)
plt.subplot(243)
plt.imshow(comp_raw_binary, cmap='gray')
plt.title('Tri-Color Difference\nComparison', y=1.1, fontsize=15)
plt.subplot(244)
plt.imshow(overlay_image)
plt.title('Normalized View of \nAffected Area', y=1.1, fontsize=15)
plt.suptitle('\n' + image_names[0][0:10] + ' Difference Analysis', y=1.025, fontsize=20)
plt.subplot(2,4, (5,8))
plt.axis('off')

columns = ('Total Area', 'Largest Area')
rows = ('Affected Percent (%):', 'Affected Area (m^2):', 'Affected Area (acres):')
cell_text = [(str(effected_raw_percent), str(largest_effected_percent)),
             (str(effected_raw_area), str(largest_effected_area)),
             (str(effected_raw_area_acres), str(largest_effected_area_acres))]

the_table = plt.table(cellText=cell_text,
                      cellLoc='center',
                      rowLabels=rows,
                      colLabels=columns,
                      bbox=[0.2, 0.35, 0.8, 0.7])
the_table.auto_set_font_size(False)
the_table.set_fontsize(15)
the_table.scale(0.75, 1.5)

plt.savefig(saved_image_dir)
#plt.show()

In [ ]:
# send email to user notifying them of the change and attach a summary image

if max_change_percent < effected_raw_percent or max_change_area_m2 < effected_raw_area:
  mail_subject = 'Change Detected ' + image_names[0][0:10]
  mail_message_body = 'A change was detected on ' + image_names[0][0:10] + '. The affected area covers ' + str(effected_raw_area) + ' square meters or ' + str(effected_raw_area_acres) + ' acres. This represents ' + str(effected_raw_percent) + ' % of the indicated area of interest. Please view the area and determine if action needs to be taken. \n \nThank you.'

  # create multipart message and set headers
  message = MIMEMultipart()
  message['From'] = MAIL_FROM
  message['To'] = MAIL_TO
  message['Subject'] = mail_subject

  # add body to email
  message.attach(MIMEText(mail_message_body, 'plain'))

  # file to attach
  filename = saved_image_dir + '.png'

  # open file in binary mode
  with open(filename, 'rb') as attachment:
    # add file as application/ octet-stream
    # email client can usually download this automatically as attachment
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())

  # encode file in ASCII characters to send by email
  encoders.encode_base64(part)

  # add header as key/value paor to attachment part
  part.add_header(
      'Content-Disposition',
      f'attachment; filename ={filename}',
  )

  # add attachment to message and convert message to string
  message.attach(part)
  text = message.as_string()

  # log in to server using secure context and send email
  context = ssl.create_default_context()
  with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as server:
    server.login(MAIL_FROM, SECURE_APP_PASSWORD)
    server.sendmail(MAIL_FROM, MAIL_TO, text)



In [ ]:
# end the program
sys.exit()